In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

In [8]:
normaliser = Normaliser()
for column in numeric_columns:
    print(data[column])
    data[column] = normaliser.standardise(data[column])
    print("after", data[column])
    break

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 0       0.723643
1      -0.886369
2      -0.632156
3      -0.208469
4       0.066928
          ...   
9995   -0.992290
9996    0.448246
9997   -0.589788
9998   -1.034659
9999   -0.907553
Name: carat, Length: 10000, dtype: float64


In [9]:
print(data)

        outcome     carat  cut  clarity  depth  table  price     x     y  \
0    -26.701232  0.723643    0        3   62.3   56.0   7948  6.73  6.70   
1      6.548093 -0.886369    1        4   60.5   59.0    898  4.69  4.66   
2      6.612562 -0.632156    2        5   60.7   58.0   1351  5.09  5.13   
3     -5.073562 -0.208469    1        5   61.2   58.0   2512  5.74  5.70   
4    -14.436557  0.066928    0        6   62.4   54.0   2751  6.01  6.08   
...         ...       ...  ...      ...    ...    ...    ...   ...   ...   
9995  10.718277 -0.992290    0        3   62.6   57.0   1002  4.42  4.40   
9996 -12.246698  0.448246    4        5   69.5   55.0   4853  6.00  5.94   
9997  11.122516 -0.589788    2        6   57.9   61.0   1273  5.28  5.33   
9998 -24.730782 -1.034659    0        0   62.0   54.0    801  4.35  4.39   
9999   8.735755 -0.907553    2        5   59.9   59.0    649  4.68  4.70   

         z  ...        b8        b9       b10  colour_G  colour_E  colour_F  \
0     4.

In [10]:
kfold_data = get_kfold_data(data=data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%



In [11]:
LGBM_HYPERPARAMETERS = {}
def objective(model_type, trial, x_train, y_train, x_val, y_val):
    parameters = {
                "objective": "regression",
                "metric": "rmse",
                "n_estimators": 100,
                "verbosity": -1,
                "bagging_freq": 1,
                "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                "seed": REPRODUCIBILITY_SEED
    }
    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse

models = {
        # "linear_regression": LinearRegression(),
        # "xgb": xgb.XGBRegressor(**HYPERPARAMETERS),
        # "random_forest": RandomForestRegressor(**HYPERPARAMETERS_2),
        # "gradient_boosting": GradientBoostingRegressor(**HYPERPARAMETERS_2),
        # "ada_boost": AdaBoostRegressor(),
        # "ridge": Ridge(),
        # "lasso": Lasso(),
        "lgbm": lgb.LGBMRegressor
        }

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]
    test_data = kfold_data[fold]["test"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    test_y = test_data["outcome"]

    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])
    test_x = test_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")

    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=30)
        
        # Train model with best hyperparameters
        best_fold_params = study.best_params
        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)

        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()
    
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
print(model_scores)

[I 2025-02-15 17:16:45,260] A new study created in memory with name: no-name-57748008-9dd0-47c2-9d70-6287b61d686b
[I 2025-02-15 17:16:45,472] Trial 0 finished with value: 9.66091751248437 and parameters: {'learning_rate': 0.04837879783536925, 'num_leaves': 95, 'subsample': 0.6533217483563118, 'colsample_bytree': 0.5450235211296038, 'min_data_in_leaf': 20}. Best is trial 0 with value: 9.66091751248437.
[I 2025-02-15 17:16:45,564] Trial 1 finished with value: 12.705619774881564 and parameters: {'learning_rate': 0.0015092169799046456, 'num_leaves': 81, 'subsample': 0.9554045793555976, 'colsample_bytree': 0.31754719081678673, 'min_data_in_leaf': 73}. Best is trial 0 with value: 9.66091751248437.
[I 2025-02-15 17:16:45,625] Trial 2 finished with value: 9.506766505453466 and parameters: {'learning_rate': 0.05408267670836075, 'num_leaves': 264, 'subsample': 0.3749267473807775, 'colsample_bytree': 0.5151647099713695, 'min_data_in_leaf': 83}. Best is trial 2 with value: 9.506766505453466.
[I 20

Fold: 1/5
Model name: lgbm
MAE: 7.6101446877814025
MSE: 92.45324711182039
RMSE: 9.615261156714382
PCC: 0.6770140206066725
Spearman R: 0.6849947001932424
R2 Score: 0.455132293663847



[I 2025-02-15 17:16:48,696] Trial 2 finished with value: 12.452674063113282 and parameters: {'learning_rate': 0.0014637980546462442, 'num_leaves': 172, 'subsample': 0.44700284745856034, 'colsample_bytree': 0.278320238976223, 'min_data_in_leaf': 65}. Best is trial 0 with value: 12.026188860193429.
[I 2025-02-15 17:16:48,781] Trial 3 finished with value: 9.435843111563091 and parameters: {'learning_rate': 0.05474901989152316, 'num_leaves': 749, 'subsample': 0.4826904897242237, 'colsample_bytree': 0.8415347660457623, 'min_data_in_leaf': 60}. Best is trial 3 with value: 9.435843111563091.
[I 2025-02-15 17:16:48,840] Trial 4 finished with value: 9.57754235621555 and parameters: {'learning_rate': 0.07821833651325703, 'num_leaves': 248, 'subsample': 0.3542704820976694, 'colsample_bytree': 0.3142091836145441, 'min_data_in_leaf': 56}. Best is trial 3 with value: 9.435843111563091.
[I 2025-02-15 17:16:48,919] Trial 5 finished with value: 9.50623490340372 and parameters: {'learning_rate': 0.07707

Fold: 2/5
Model name: lgbm
MAE: 7.60311944839308
MSE: 91.25374364377954
RMSE: 9.552682536532842
PCC: 0.6566509988831946
Spearman R: 0.6548195780545227
R2 Score: 0.4280381668143587



[I 2025-02-15 17:16:52,974] Trial 1 finished with value: 9.378140184025971 and parameters: {'learning_rate': 0.07902969163491262, 'num_leaves': 806, 'subsample': 0.6655370900112086, 'colsample_bytree': 0.7816331538715336, 'min_data_in_leaf': 76}. Best is trial 1 with value: 9.378140184025971.
[I 2025-02-15 17:16:53,046] Trial 2 finished with value: 9.41297126276443 and parameters: {'learning_rate': 0.022115164871828387, 'num_leaves': 636, 'subsample': 0.5797466290917482, 'colsample_bytree': 0.800978590746472, 'min_data_in_leaf': 78}. Best is trial 1 with value: 9.378140184025971.
[I 2025-02-15 17:16:54,528] Trial 3 finished with value: 10.995378787976966 and parameters: {'learning_rate': 0.005035896241766604, 'num_leaves': 802, 'subsample': 0.8334982171146725, 'colsample_bytree': 0.8291780007183888, 'min_data_in_leaf': 2}. Best is trial 1 with value: 9.378140184025971.
[I 2025-02-15 17:16:54,809] Trial 4 finished with value: 12.220733376833389 and parameters: {'learning_rate': 0.001269

Fold: 3/5
Model name: lgbm
MAE: 7.4854626676254385
MSE: 87.04926244597897
RMSE: 9.330019423665686
PCC: 0.6843288241125911
Spearman R: 0.6942068922683173
R2 Score: 0.46627775802768



[I 2025-02-15 17:16:57,373] Trial 0 finished with value: 10.473559417353982 and parameters: {'learning_rate': 0.00712219475046953, 'num_leaves': 517, 'subsample': 0.9051004888297339, 'colsample_bytree': 0.7767630538587638, 'min_data_in_leaf': 32}. Best is trial 0 with value: 10.473559417353982.
[I 2025-02-15 17:16:57,466] Trial 1 finished with value: 12.552744565635091 and parameters: {'learning_rate': 0.002577609564833178, 'num_leaves': 401, 'subsample': 0.40647493102835136, 'colsample_bytree': 0.07844585678935852, 'min_data_in_leaf': 36}. Best is trial 0 with value: 10.473559417353982.
[I 2025-02-15 17:16:57,570] Trial 2 finished with value: 10.34760413892291 and parameters: {'learning_rate': 0.006889973122831439, 'num_leaves': 999, 'subsample': 0.32933019953997056, 'colsample_bytree': 0.9070475276388318, 'min_data_in_leaf': 35}. Best is trial 2 with value: 10.34760413892291.
[I 2025-02-15 17:16:57,728] Trial 3 finished with value: 10.15008377721305 and parameters: {'learning_rate': 

Fold: 4/5
Model name: lgbm
MAE: 7.36846870515466
MSE: 86.48528725364643
RMSE: 9.299746623088526
PCC: 0.6799199968702488
Spearman R: 0.6867490651363536
R2 Score: 0.46219495481382655



[I 2025-02-15 17:17:01,091] Trial 3 finished with value: 10.45461992006161 and parameters: {'learning_rate': 0.01640432153008423, 'num_leaves': 962, 'subsample': 0.7549998815152686, 'colsample_bytree': 0.32722982625594843, 'min_data_in_leaf': 37}. Best is trial 2 with value: 10.364111376410529.
[I 2025-02-15 17:17:01,153] Trial 4 finished with value: 9.407242713348587 and parameters: {'learning_rate': 0.04936542615732432, 'num_leaves': 543, 'subsample': 0.147948042551582, 'colsample_bytree': 0.4922714887305508, 'min_data_in_leaf': 27}. Best is trial 4 with value: 9.407242713348587.
[I 2025-02-15 17:17:01,594] Trial 5 finished with value: 12.317921150349074 and parameters: {'learning_rate': 0.0012463408303853125, 'num_leaves': 337, 'subsample': 0.847322088681242, 'colsample_bytree': 0.5836573356657384, 'min_data_in_leaf': 10}. Best is trial 4 with value: 9.407242713348587.
[I 2025-02-15 17:17:02,250] Trial 6 finished with value: 11.664534374747136 and parameters: {'learning_rate': 0.002

Fold: 5/5
Model name: lgbm
MAE: 7.428196092474504
MSE: 86.66497509726683
RMSE: 9.309402510218732
PCC: 0.6879123552099282
Spearman R: 0.6897365213814538
R2 Score: 0.4696232604479347

{'lgbm': defaultdict(<class 'list'>, {'mae': 7.499078320285818, 'mse': 88.78130311049844, 'rmse': 9.421422450044034, 'pcc': np.float64(0.677165239136527), 'spearman_r': np.float64(0.682101351406778), 'r2_score': np.float64(0.45625328675352933)})}
